# *Ejecutad esto primero*

In [ ]:
!pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


import requests
import pandas as pd
import datetime
import time

# ---------------- CONFIGURACIÓN ----------------
# PEGA AQUÍ TU TOKEN (mantén las comillas)
TOKEN = "8535a75306ed2ab3a0f502678654ab36a12f0884cae56c68f5e5990142e63c36" 

# Identificador de "Demanda Real" en ESIOS
INDICATOR_ID = "1293" 

# Cabeceras para la autenticación
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": TOKEN
}

def get_esios_data(start_date, end_date):
    """Función para pedir datos a la API en un rango de fechas"""
    url = f"https://api.esios.ree.es/indicators/{INDICATOR_ID}?start_date={start_date}&end_date={end_date}"
    
    print(f"Descargando datos desde {start_date} hasta {end_date}...")
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        # La información importante está en ['indicator']['values']
        values = data['indicator']['values']
        return values
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# ---------------- PROCESO PRINCIPAL ----------------
all_data = []

# Iteramos por años para no saturar la API
years = range(2015, 2026) # De 2015 a 2025

for year in years:
    start = f"{year}-01-01T00:00:00"
    end = f"{year}-12-31T23:59:59"
    
    # Descargar
    yearly_data = get_esios_data(start, end)
    all_data.extend(yearly_data)
    
    # Pequeña pausa para ser "amables" con el servidor
    time.sleep(1)

# ---------------- CREAR DATAFRAME Y GUARDAR ----------------
print(f"\nProcesando {len(all_data)} registros...")

# Convertimos la lista de diccionarios a DataFrame de Pandas
df = pd.DataFrame(all_data)

# Filtramos solo las columnas que nos interesan (Valor y Fecha)
# 'value' es la demanda en MWh
# 'datetime' es la fecha y hora
# 'geo_id' suele ser 8741 (Península), a veces hay Canarias/Baleares. 
# Para este ejercicio simplificado, nos quedaremos con todo y luego filtraremos.
if not df.empty:
    df = df[['datetime', 'value', 'geo_name']]
    
    # Renombrar columnas para que sea más limpio
    df.columns = ['fecha', 'demanda', 'region']
    
    # Guardar en CSV
    file_name = "demanda_electrica_esios_2015_2025.csv"
    df.to_csv(file_name, index=False)
    
    print(f"¡ÉXITO! Se ha guardado el archivo: {file_name}")
    print(f"Total de filas descargadas: {df.shape[0]}")
    print(df.head())
else:
    print("No se han podido descargar datos. Revisa tu Token.")

# **0. No volver a Usar**

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# --- CONFIGURACIÓN ---
TOKEN = "8535a75306ed2ab3a0f502678654ab36a12f0884cae56c68f5e5990142e63c36" 
INDICATOR_ID = "1293" 

headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": TOKEN
}

def get_monthly_data(year, month):
    """Pide datos de un mes específico para evitar Timeouts"""
    # Calcular fechas de inicio y fin de mes
    start_date = f"{year}-{month:02d}-01T00:00:00"
    
    # Cálculo del último día del mes
    if month == 12:
        next_month = datetime(year + 1, 1, 1)
    else:
        next_month = datetime(year, month + 1, 1)
        
    end_date_dt = next_month - timedelta(seconds=1)
    end_date = end_date_dt.strftime("%Y-%m-%dT%H:%M:%S")

    # Pedimos específicamente geoid=8741 (Península) para reducir volumen
    url = f"https://api.esios.ree.es/indicators/{INDICATOR_ID}?start_date={start_date}&end_date={end_date}&geo_ids[]=8741"
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code == 200:
            data = response.json()
            return data['indicator']['values']
        else:
            print(f" Error {response.status_code} en {month}/{year}")
            return []
    except Exception as e:
        print(f" Excepción en {month}/{year}: {e}")
        return []

# --- PROCESO DE DESCARGA ---
all_data = []
years = range(2015, 2025) # De 2015 a 2024 (10 años completos)

print("Iniciando descarga robusta (mes a mes)...")

for year in years:
    for month in range(1, 13):
        # Imprime progreso en la misma línea
        print(f"Procesando: {month:02d}/{year}...", end="\r")
        
        month_data = get_monthly_data(year, month)
        
        if month_data:
            all_data.extend(month_data)
        else:
            # Reintento simple si falla una vez
            time.sleep(2)
            month_data = get_monthly_data(year, month)
            if month_data:
                all_data.extend(month_data)
            else:
                print(f"\nFallo definitivo en {month}/{year}")

        # Pausa de cortesía para la API
        time.sleep(0.3)

print(f"\n\n¡Descarga finalizada!")

# --- GUARDADO FINAL ---
if all_data:
    df = pd.DataFrame(all_data)
    
    # Seleccionamos y renombramos
    # La API a veces devuelve columnas distintas, aseguramos las básicas
    if 'datetime' in df.columns and 'value' in df.columns:
        df = df[['datetime', 'value', 'geo_name']]
        df.columns = ['fecha', 'demanda', 'region']
        
        # Guardamos el CSV definitivo
        file_name = "demanda_electrica_peninsula_2015_2024.csv"
        df.to_csv(file_name, index=False)
        
        print(f"✅ Archivo guardado correctamente: {file_name}")
        print(f"Total de filas: {len(df)}")
        print("Muestra de datos:")
        print(df.head())
    else:
        print("Error: Las columnas esperadas no están en la respuesta.")
        print(df.columns)
else:
    print("No se han descargado datos. Revisa el Token.")

Iniciando descarga robusta (mes a mes)...
Procesando: 12/2024...

¡Descarga finalizada!
✅ Archivo guardado correctamente: demanda_electrica_peninsula_2015_2024.csv
Total de filas: 663288
Muestra de datos:
                           fecha  demanda     region
0  2015-01-01T00:00:00.000+01:00  25459.0  Península
1  2015-01-01T00:10:00.000+01:00  25591.0  Península
2  2015-01-01T00:20:00.000+01:00  25531.0  Península
3  2015-01-01T00:30:00.000+01:00  25453.0  Península
4  2015-01-01T00:40:00.000+01:00  25329.0  Península


# **1. Carga de datos a día 10.12**

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import os
import sqlite3

# Nombre del archivo
file_name = "demanda_electrica_peninsula_2015_2024.csv"

# Verifica si el archivo existe
if os.path.exists(file_name):
    
    # Carga el CSV
    df = pd.read_csv(file_name)
    
    # IMPORTANTE: Al leer de CSV, las fechas las lee como texto. 
    # Hay que convertirlas a datetime siempre que cargues.
    if 'fecha' in df.columns:
        # Convierte a datetime asegurando que sea UTC
        df['fecha'] = pd.to_datetime(df['fecha'], utc=True)
        
        # Usamos .dt.tz_convert() en lugar de convert_timezone ya que esta función está deprecada
        df['fecha'] = df['fecha'].dt.tz_convert('Europe/Madrid')
    
    print(f"Tenemos {len(df)} filas listas para trabajar.")
    print(df.head(3))
    
else:
    print("NO se encuentra el archivo CSV.")
    print("Hay que pedir otra clave, la hemos liao.")

Tenemos 663288 filas listas para trabajar.
                      fecha  demanda     region
0 2015-01-01 00:00:00+01:00  25459.0  Península
1 2015-01-01 00:10:00+01:00  25591.0  Península
2 2015-01-01 00:20:00+01:00  25531.0  Península


In [ ]:
# Carga el CSV  en Python
file_name = "demanda_electrica_peninsula_2015_2024.csv"
df = pd.read_csv(file_name)

# Conecta a la base de datos (se crea sola si no existe)
# Se creará un archivo llamado 'proyecto_energia.db' si no existía
conn = sqlite3.connect("proyecto_energia.db")
cursor = conn.cursor()

# Crea la tabla SQL 

create_table_query = """
CREATE TABLE IF NOT EXISTS demanda_electrica (
    fecha DATETIME PRIMARY KEY,
    demanda REAL,
    region TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()
print("Base de datos y tabla creadas correctamente.")

# Del df a SQL
# 'if_exists="replace"' asegura que si lo corremos dos veces, no dupliques datos
# 'index=False' porque la fecha ya es una columna, no el índice
df.to_sql("demanda_electrica", conn, if_exists="replace", index=False)

print(f"Se han insertado {len(df)} filas en la base de datos 'proyecto_energia.db'.")

Base de datos y tabla creadas correctamente.
Se han insertado 663288 filas en la base de datos 'proyecto_energia.db'.


# **2. Prueba de SQL**

In [14]:
print("\n¿Cuándo se han producido los 5 mayores consumos en los últimos 10 años?")
query = """
SELECT fecha, demanda 
FROM demanda_electrica 
ORDER BY demanda DESC 
LIMIT 5;
"""
resultado = pd.read_sql(query, conn)
print(resultado)

# Cerramos conexión
conn.close()


¿Cuándo se han producido los 5 mayores consumos en los últimos 10 años?
                           fecha  demanda
0  2021-01-08T13:20:00.000+01:00  42052.0
1  2021-01-08T13:30:00.000+01:00  41960.0
2  2021-01-08T13:10:00.000+01:00  41885.0
3  2021-01-08T13:40:00.000+01:00  41867.0
4  2021-01-12T20:40:00.000+01:00  41699.0
